In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras import regularizers

# Define constants
NUM_CLASSES = 4  # Number of classes (correct, partially correct, incorrect, none)
IMG_WIDTH, IMG_HEIGHT = 224, 224  # Input image dimensions for MobileNetV2
BATCH_SIZE = 32
EPOCHS = 50

# Load MobileNetV2 pre-trained model (include_top=False to exclude the classification layers)
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(IMG_WIDTH, IMG_HEIGHT, 3))

# Unfreeze the last few layers of the base model for fine-tuning
for layer in base_model.layers[-30:]:
    layer.trainable = True

# Add custom classification layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)
x = Dropout(0.5)(x)  # Adjusted dropout rate
predictions = Dense(NUM_CLASSES, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Implement a learning rate schedule
lr_schedule = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Create data generators for training and validation with data augmentation
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = train_datagen.flow_from_directory(
    "/home/adarsh/Desktop/yoga/train/",
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

validation_datagen = ImageDataGenerator(rescale=1.0 / 255)

validation_generator = validation_datagen.flow_from_directory(
    "/home/adarsh/Desktop/yoga/test/",
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

# Define early stopping
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model with early stopping and learning rate schedule
model.fit(
    train_generator,
    steps_per_epoch=train_generator.n // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=validation_generator,
    validation_steps=validation_generator.n // BATCH_SIZE,
    callbacks=[early_stop, lr_schedule]  # Add early stopping and LR schedule to the callbacks
)

# Save the model for later use
model.save('pose_classification_model_mobilenetv2.h5')


Found 24855 images belonging to 4 classes.
Found 5636 images belonging to 4 classes.
Epoch 1/50
776/776 [==============================] - 658s 841ms/step - loss: 0.9068 - accuracy: 0.9509 - val_loss: 2.5528 - val_accuracy: 0.6317 - lr: 0.0010
Epoch 2/50
776/776 [==============================] - 653s 841ms/step - loss: 0.0944 - accuracy: 0.9807 - val_loss: 0.5791 - val_accuracy: 0.8130 - lr: 0.0010
Epoch 3/50
776/776 [==============================] - 651s 839ms/step - loss: 0.1295 - accuracy: 0.9751 - val_loss: 34.3687 - val_accuracy: 0.2786 - lr: 0.0010
Epoch 4/50
776/776 [==============================] - 647s 834ms/step - loss: 0.0540 - accuracy: 0.9905 - val_loss: 7.0651 - val_accuracy: 0.5009 - lr: 0.0010
Epoch 5/50
776/776 [==============================] - 650s 837ms/step - loss: 0.0905 - accuracy: 0.9830 - val_loss: 11.1480 - val_accuracy: 0.4819 - lr: 0.0010
Epoch 6/50
776/776 [==============================] - 644s 830ms/step - loss: 0.0225 - accuracy: 0.9952 - val_loss: 0.

/home/adarsh/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
